## João Pedro Rodrigues Freitas - 11316552

### TODO
- Fazer os mini batches (train, test)
- Fazer o one-hot

In [1339]:
import numpy as np
import matplotlib.pyplot as plt

## Data
Esta classe realiza o pre-processamento dos dados, permitindo:
- Permite definir uma porcentagem de dados para treino
- Embaralhar as linhas
- Normalizar os dados (para cada coluna, de acordo com o max e min de cada coluna)
- Obter o conjunto de treino
- Obter o conjunto de teste
- Obter o número de atributos (numero de colunas)

In [1340]:
class Data():
    def __init__(self, X: np.ndarray) -> None:
        self.data = X


    def getData(self) -> np.ndarray:
        return self.data
    

    def setTrainRatio(self, ratio: float) -> None:
        self.train_ratio = ratio


    def getTrainRatio(self) -> float:
        return self.train_ratio
    
    
    def getAttrSize(self) -> int:
        return self.data.shape[1]
    
    
    def shuffleData(self) -> None:
        self.data = np.random.permutation(self.data)


    def getTrainData(self) -> np.ndarray:
        return self.data[:int(self.data.shape[0] * self.train_ratio)]
    
    
    def getTestData(self) -> np.ndarray:
        return self.data[int(self.data.shape[0] * self.train_ratio):]
    

    def normalize(self) -> None:
        '''Normaliza cada atributo para o intervalo [0, 1]'''
        for i in range(self.getAttrSize()):
            self.data[:,i] = (self.data[:,i] - np.min(self.data[:,i])) / (np.max(self.data[:,i]) - np.min(self.data[:,i]))

            

## Layer
Os layers contém as informações das camadas.

Os pesos e bias são inicializados de acordo com uma distribuição uniforme
no intervalo de -0.5 a 0.5

O layer permite realizar o forward de si próprio. 

In [1341]:
class Layer():
    def __init__(self, n_neurons: int, actv_func, inputs = None, lastLayer: bool = False) -> None:
        self.n_neurons = n_neurons
        self.actv_func = actv_func

        self.W = None
        self.b = None
        self.out = None # Saída da camada
        self.actv = None # Saída com função de ativação

        self.inputs = inputs # Entradas da camada
        self.shape = None
        self.inputShape = None

        self.lastLayer = lastLayer


    def _setWeights(self) -> None:
        if self.W is None and self.inputs is not None:
            self.inputShape = self.inputs.shape
            self.shape = (self.inputShape[-1], self.n_neurons) # Shape da matriz de pesos a ser criada

            # Inicializa os pesos e bias com distribuição uniforme
            # entre -0.5 e 0.5
            self.W = np.random.rand(self.shape[0], self.shape[1]) - 0.5
            self.b = np.random.rand(self.n_neurons) - 0.5


    # Realiza o forward da camada
    def process(self, inputs) -> None:
        self.inputs = inputs

        self._setWeights() # Inicializa os pesos e bias, caso n tenham sido inicializados

        self.out = np.dot(self.inputs, self.W) + self.b

        # Se for a última camada, não aplica a função de ativação
        if not self.lastLayer:
            self.actv = self.actv_func(self.out)
        else:
            self.actv = self.out


    def getOutput(self):
        return self.out
    
    
    def getActivation(self):
        return self.actv
        


# Autoencoder

In [1342]:
class Autoencoder():
    def __init__(self) -> None:
        self.layers = np.array([], dtype=Layer)

        self.epochs = None
        self.lr = None

        self.inputs = None # X
        self.targets = None # X

        self.outputs = []
        self.ativations = [] # X_Hat

    def addLayer(self, layer: Layer) -> None:
        self.layers = np.append(self.layers, layer)

    # TODO: separar entre encoder e decoder
    def forward(self, inputs):
        self.ativations = []
        self.outputs = []

        self.ativations.append(inputs)
        self.outputs.append(inputs)

        for layer in self.layers:
            layer.process(inputs)
            inputs = layer.getActivation()
            output = layer.getOutput()

            self.outputs.append(output)
            self.ativations.append(inputs)

        return inputs
    
    # TODO: separar entre encoder e decoder
    def backward(self) -> None:
        i = self.layers.size
        n = self.inputs.shape[0]

        err = 2 * (self.targets - self.ativations[-1]) # x - x_hat

        for layer, z in zip(self.layers[::-1], self.outputs[::-1]):
            delta = err

            if (i != self.layers.size):
                delta *= layer.actv_func(z, derivative=True)

            err = np.dot(delta, layer.W.T) # Atualiza o erro pro layer anterior

            if i > 0:
                prev_ativ = self.ativations[i-1]
                dW = np.dot(prev_ativ.T, delta) / n
                db = delta.mean()

                layer.W += self.lr * dW
                layer.b += self.lr * db


            i -= 1
        
    def fit(self, inputs, targets, lr: float = 0.01, epochs: int = 100) -> None:
        self.inputs = inputs
        self.targets = targets
        self.lr = lr

        for epoch in range(epochs):
            self.forward(inputs)
            self.backward()

            if epoch % 10 == 0:
                print(f'Epoch: {epoch}')
                print(f'Loss: {np.mean(np.square(self.targets - self.ativations[-1]))}')


    def predict(self, inputs) -> np.ndarray:
        self.forward(inputs)
        return self.ativations[-1]

## Função de ativação

Foi utilizada a função de ativação sigmoide.

A função dada por 1 / (1 + np.exp(-Z)) é sensível a overflow. Desse modo, ela
foi adaptada (mesma função do pytorch).

In [1343]:
def sigmoid(x, derivative=False):
    if derivative:
        return sigmoid(x) * (1 - sigmoid(x))

    positives = x >= 0
    negatives = ~positives
    
    exp_x_neg = np.exp(x[negatives])
    
    y = x.copy()
    y[positives] = 1 / (1 + np.exp(-x[positives]))
    y[negatives] = exp_x_neg / (1 + exp_x_neg)
    
    return y

## Criação do modelo
Esta função instancia a classe autoencoder.

Ela recebe a dimensão dos atributos de entrada, as camadas ocultas (na parte do
encoder) e a função de ativação.

Esta função irá instanciar os layers de forma que seja espelhado, ou seja:

Se inputDim = 10, e as camadas ocultas têm os tamanhos layers=[5, 3], então
a camada com 3 neurônios será a dimensão latente, sendo tudo antes dela espelhado.

Assim, ficará no formato: [10, 5, 3, 5, 10].

In [1344]:
def createModel(inputDim, layers, actv_func):
    model = Autoencoder()

    # Adiciona as camadas do encoder
    for i in range(len(layers)):
        model.addLayer(Layer(layers[i], actv_func))

    # Adiciona as camadas do decoder
    for i in range(len(layers)-2, -1, -1):
        model.addLayer(Layer(layers[i], actv_func))

    # Adiciona a camada de saída
    model.addLayer(Layer(inputDim, actv_func, lastLayer = True))

    return model

## Main

In [1345]:
X = np.vstack([np.random.normal(1, 0.5, [100,64]),
                np.random.normal(-2, 1, [100,64]),
                np.random.normal(3, 0.75, [100,64])])

data = Data(X)

data.normalize()
data.shuffleData()
data.setTrainRatio(0.8)

inputDim = data.getAttrSize()

# Número de neurônios em cada camada escondida
# A última camada dessa lista é a dimensão latente
# O espelhamento é feito automaticamente
# Exemplo:
# Para um X com 64 atributos, se hiddenLayers = [32, 16], a rede terá a seguinte estrutura:
# [64, 32, 16, 32, 64]
# Em que o primeiro valor é a dimensão de entrada e o último é a dimensão de saída
hiddenLayers = [32, 16]

autoencoder = createModel(inputDim, hiddenLayers, actv_func=sigmoid)

# autoencoder = createModel([inputDim] + teste + [inputDim], actv_func=sigmoid)
autoencoder.fit(data.getTrainData(), data.getTrainData(), lr=0.01, epochs=100)

Epoch: 0
Loss: 1.3487445745223747
Epoch: 10
Loss: 0.08098388132229166
Epoch: 20
Loss: 0.054305783297435364
Epoch: 30
Loss: 0.053412814099941315
Epoch: 40
Loss: 0.05328875247362576
Epoch: 50
Loss: 0.053187049884205556
Epoch: 60
Loss: 0.053083641615765335
Epoch: 70
Loss: 0.0529775013871444
Epoch: 80
Loss: 0.052868373738460255
Epoch: 90
Loss: 0.05275602434467895


# Testar o modelo

In [1346]:
X = data.getTestData()
X_hat = autoencoder.predict(X)

err = np.mean(np.square(X - X_hat))

print(f'Erro quadrático médio: {err * 100:.3f}%')

Erro quadrático médio: 5.492%
